# Predicting enrollment for bank customers using AWS sageMaker

## Objective :

The main objective is to develop a machine learning model to predict whether a customer will enroll for a certificate of deposit.

### Importing Libraries 

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import Session, s3_input

#### Here, we specify bucketname and region.

In [20]:
bucketname = 'machine-learning-2412'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


#### We create a s3 bucket. 

In [21]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucketname)
    print("S3 bucket created sucessfully")
except Exception as e:
    print("S3 error: " + str(e))

S3 bucket created sucessfully


#### Specifying output path

In [22]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucketname,prefix)
print(output_path)

s3://machine-learning-2412/xgboost-as-a-built-in-algo/output


### Download dataset and store it into S3 bucket. 

In [23]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


#### Spliting train and test data. 

In [25]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


#### To use an Amazon SageMaker pre-built XGBoost model, we will need to reformat the header and first column of the training data. We then stores training data to S3 bucket.

In [28]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketname).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucketname, prefix), content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


#### Similar to training data, we reformat headers and first column of the testing data. We then stores training data to S3 bucket.

In [30]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucketname).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucketname, prefix), content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


#### To use XGBoost built in algorithm, we need to call get_image_uri method. 

In [31]:
container = get_image_uri(boto3.Session().region_name,'xgboost', repo_version='1.0-1')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


#### Here, we specify hyperparameters. 

In [32]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

#### Next, we need to set up the Amazon SageMaker session and create an instance of the XGBoost model (an estimator)

In [33]:
estimator = sagemaker.estimator.Estimator(image_name=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


#### Now, we will train our model. 

In [34]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2020-10-07 08:56:12 Starting - Starting the training job...
2020-10-07 08:56:13 Starting - Launching requested ML instances......
2020-10-07 08:57:37 Starting - Preparing the instances for training...
2020-10-07 08:58:10 Downloading - Downloading input data...
2020-10-07 08:58:23 Training - Downloading the training image...
2020-10-07 08:59:04 Uploading - Uploading generated training model
2020-10-07 08:59:04 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[08:58:53] 28831x59 matrix with 1

#### Next we will deploy the trained model to an endpoint, reformat then load the CSV data, then run the model to create predictions.

In [35]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

#### Next,we will predict whether customers in the test data enrolled for the bank product or not

In [41]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions, sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [42]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

#### In this step, we will evaluate the performance and accuracy of the machine learning model by creating confusion martix.

In [43]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



#### We can see that out model is 89.7% accurate. From the confision matrix gives information on how many percentage is predicted result is matched with actual data. 

#### Here, we delete Amazon SageMaker endpoint and the objects in our S3 bucket so it does not cost us more money. 

In [48]:
bucket_to_delete = boto3.resource('s3').Bucket(bucketname)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'AH4XAP8K7N9S0K4M',
   'HostId': 'mqou61Z2Z948OYe1THsfQEIxl49y2fpHpjW3GtE5Vyw1pxplVot9f8R2fMjQDt5jde+o5+WIbws=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'mqou61Z2Z948OYe1THsfQEIxl49y2fpHpjW3GtE5Vyw1pxplVot9f8R2fMjQDt5jde+o5+WIbws=',
    'x-amz-request-id': 'AH4XAP8K7N9S0K4M',
    'date': 'Wed, 07 Oct 2020 09:21:18 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/test/test.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2020-10-07-08-56-11-845/output/model.tar.gz'},
   {'Key': 'xgboost-as-a-built-in-algo/train/train.csv'}]}]